<a href="https://colab.research.google.com/github/ARISE-NYU/diggs/blob/pedro-dev/data/eda_notebooks/2022_05_11_did_processed_data_EDA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Import Data/Housekeeping

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import seaborn as sns
import matplotlib as mpl


# 1. Import Data
## 1.1 Read Data

In [25]:
# os.makedirs()
project_directory_path = "<YOUR PATH>"
did_data_path = os.path.join(project_directory_path, "<YOUR PATH>")

In [5]:
#aride df
did_df = pd.read_csv(did_data_path)
did_df_working = did_df.copy()
did_df_working['std_family_income_range'] = did_df_working['orig_nondegree_application_family_income_range']

did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='Less than $19,999'] = '1 - Less than $19,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='$20,000 to 39,999'] = '2 - $20,000 to 39,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='$40,000 to 59,999'] = '3 - $40,000 to 59,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='$60,000 to 79,999'] = '4 - $60,000 to 79,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='$80,000 to 99,999'] = '5 - $80,000 to 99,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='$100,000 to 149,999'] = '6 - $100,000 to 149,999'
did_df_working['std_family_income_range'].loc[did_df_working['std_family_income_range']=='More than $150,000'] = '7 - More than $150,000'

did_df_complete = did_df_working[did_df_working.std_decision.isin(['matriculated','deny'])]
did_df_matriculated = did_df_working[did_df_working.std_decision.isin(['matriculated'])]
did_df_deny = did_df_working[did_df_working.std_decision.isin(['deny'])]
did_df_no_decision = did_df_working[did_df_working.std_decision.isin(['no_decision'])]


# did_df_aa = did_df_working[did_df_working.std.race.isin(['african'])]
# did_df_working.std.race.value_counts()
did_df_working.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 307 columns):
 #    Column                                                  Dtype  
---   ------                                                  -----  
 0    orig_unique_identifier                                  int64  
 1    orig_nondegree_application_sis_decision                 object 
 2    orig_app_started_date                                   object 
 3    orig_app_submitted_date                                 object 
 4    orig_race                                               object 
 5    orig_hispanic                                           object 
 6    orig_nondegree_application_cultural_identity            object 
 7    orig_nondegree_application_cultural_identity_other      object 
 8    orig_sex                                                object 
 9    orig_gender_identity                                    object 
 10   orig_gender_identity_description              

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Function

In [6]:
def plot_pie(did_df,attribute,title = None):    
    fig, ax = plt.subplots(figsize=(6, 3), subplot_kw=dict(aspect="equal"))
    did_df = did_df.sort_values(attribute)

    values = did_df[attribute].value_counts(dropna = False)
    labels = did_df[attribute].value_counts(dropna = False).index#.sort_values()
    explode = [.5 if value/values.sum() <.01 else 0 for value in values]


    wedges, autotexts,  = ax.pie(values, 
                                   explode=explode, 
    #                                labels=y, 
    #                                autopct='%1.1f%%',
    #                                shadow=True, 
    #                                startangle=90
    #                                pctdistance = 1.5
                                  )

    ax.axis('equal')

    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")

    for i, p in enumerate(wedges):
        if explode[i] == 0:
            ang = (p.theta2 - p.theta1)/2. + p.theta1
            y = np.sin(np.deg2rad(ang))
            x = np.cos(np.deg2rad(ang))
            horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
            connectionstyle = "angle,angleA=0,angleB={}".format(ang)
            kw["arrowprops"].update({"connectionstyle": connectionstyle})

            ax.annotate("{}%".format(np.round(values[i]/values.sum()*100,1)), xy=(x, y), xytext=(1.5*np.sign(x), 1.5*y),
                        horizontalalignment=horizontalalignment, **kw)

    ax.legend(wedges, labels,
              title=attribute,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1)
             )

    plt.setp(autotexts, size=8, weight="bold")

    if title == None:
      plt.title('{} Distribution'.format(attribute), y = 1.2)
      plt.show()
    else:
      plt.title(title, y = 1.2)
      plt.show() 

    return values, labels, explode

def make_autopct(values,thresh = 2):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        if pct>thresh:
          return '{p:.2f}%\n({v:d})'.format(p=pct,v=val)
        else:
          return ''
    return my_autopct

# OutComes EDA


In [7]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

In [10]:
attribute = 'std_decision'
title_attribute ='Decision'
title = 'Decision Distribution Across Full Dataset'


fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(aspect="equal"))


did_df = did_df.sort_values(attribute)
values = did_df[attribute].value_counts(dropna = False)
labels = did_df[attribute].value_counts(dropna = False).index#.sort_values()
labels = [label.replace('_',' ').title() for label in labels]

explode = [.2 if value/values.sum() <.01 else 0 for value in values]

palette_color = sns.color_palette('pastel')
mpl.rcParams['font.size'] = 12.0

wedges, autotexts, _ = ax.pie(values, 
                             explode=explode,
                             colors = palette_color,
#                                labels=y, 
                               autopct=make_autopct(values,thresh = 2),
#                                shadow=True, 
#                                startangle=90
#                                pctdistance = 1.5
                              )
# ax.axis('equal')

# bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
# kw = dict(arrowprops=dict(arrowstyle="-",alpha=.5),
#           bbox=bbox_props, zorder=0, va="center")

# for i, p in enumerate(wedges):
#     if explode[i] == 0:
#       ang = (p.theta2 - p.theta1)/2. + p.theta1
#       y = np.sin(np.deg2rad(ang))
#       x = np.cos(np.deg2rad(ang))
#       horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
#       connectionstyle = "angle,angleA=0,angleB={}".format(ang)
#       kw["arrowprops"].update({"connectionstyle": connectionstyle})

#       ax.annotate("{}%".format(np.round(values[i]/values.sum()*100,1)), xy=(x, y), xytext=(1.5*np.sign(x), 1.5*y),
#                   horizontalalignment=horizontalalignment, **kw)
      
#     else:
#       ang = (p.theta2/1 - p.theta1)/1. + p.theta1
#       y = np.sin(np.deg2rad(ang))
#       x = np.cos(np.deg2rad(ang))
#       horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
#       connectionstyle = "angle,angleA=0,angleB={}".format(ang)
#       kw["arrowprops"].update({"connectionstyle": connectionstyle})

#       ax.annotate("{}%".format(np.round(values[i]/values.sum()*100,1)), xy=(x, y), xytext=(2*np.sign(x), 2*y),
#                   horizontalalignment=horizontalalignment, **kw)

ax.legend(wedges, labels,
          title=title_attribute,
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1)
          )

plt.setp(autotexts, size=8, weight="bold")


if title == None:
  plt.title('{} Distribution'.format(attribute), y = 1.2)
  plt.close()# plt.show()
else:
  # plt.title(title, y = 1.,fontsize=20)
  plt.close()#plt.show() 

In [12]:
fig, ax = plt.subplots(1,figsize=(10,10), sharey = True)
decision_counts = pd.DataFrame(did_df.std_decision.value_counts().reset_index())

decision_counts['Decision Status'] = decision_counts['index'].apply(lambda row: row.replace('_',' ').title() if isinstance(row,str) else row)


sns.barplot(x='Decision Status',
                 y='std_decision',
                 data= decision_counts,
                 ax = ax,
                 palette = palette_color)


ax.set_title('Application Decision Counts, Total = 1330',size = 20)
ax.set_xlabel('', y = -23)
ax.set_ylabel('Counts')
sns.despine()

plt.close()#plt.show() 

In [13]:
decision_counts
decision_counts['Decision Status'] = decision_counts['index'].apply(lambda row: row.replace('_',' ').title() if isinstance(row,str) else row)
# decision_melt
# decision_counts
# sorted_hue = list(decision_melt[title_feature].value_counts().sort_index().index)

In [14]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    # did_df_deny,
                    did_df_matriculated,
                    ]

outcomes_title_list = ['All Started Applications',
                      'Incomplete Applications',
                      'Complete Applications',
                      # 'Denied Applications',
                      'Accepted Applications',
                      ]
fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

axes = np.ravel(ax)
attribute = 'std_race'
title_atribute = 'Race'
legend_labels = []
for i,ax in enumerate(axes):
  working_df = outcomes_df_list[i]

  working_df = working_df.sort_values(attribute)
  values_series = working_df[attribute].value_counts(dropna = False).sort_index()
  values = np.array(values_series.values)
  labels = [i.replace('_',' ').title() if isinstance(i,str) else i for i in values_series.index ]


  explode = [.5 if value/values.sum() <.02 else 0 for value in values]
  pct_labels = [s + '%' if explode[i]==0 else '' for i,s in enumerate(list(np.round(values/values.sum()*100,1).astype(str)))]

  palette_color = sns.color_palette('pastel')
  mpl.rcParams['font.size'] = 10
  wedges, autotexts,  _= ax.pie(values, 
                              explode=explode,
                              colors = palette_color,
                              # labels = labels,
                              autopct=make_autopct(values)#pct_labels,#'%1.1f%%',                               
                              )

  title = '{}'.format(outcomes_title_list[i])
  ax.set_title(title)
  
  print(len(legend_labels))
  if len(labels)>len(legend_labels):
    # print(len(legend_labels))
    legend_labels = labels
    legend_wedges = wedges

plt.legend(legend_wedges, legend_labels,
          title=title_atribute,
          loc="lower right",
          bbox_to_anchor=(1, -.2, .2, 0)
          )
plt.close()#plt.show() 

0
6
6
6


In [15]:
outcome_series_list = []
feature = 'std_race'
title_feature = 'Race'
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

for df in outcomes_df_list:
  total_counts = pd.Series({'total count':df[feature].value_counts(dropna=False).sum()})
  normalized_counts = df[feature].value_counts(dropna=False)/df[feature].value_counts(dropna=False).sum()
  # normalized_counts = normalized_counts.append(total_counts)
  outcome_series_list.append(normalized_counts)

decision_contingency = pd.DataFrame(outcome_series_list,index = ['all','incomplete','complete','deny','matriculated']).T.reset_index()
decision_contingency = decision_contingency.rename(columns = {'index':feature})
decision_contingency['Incomplete vs. All'] = np.log10(decision_contingency['incomplete']/decision_contingency['all'])
decision_contingency['Complete vs. All'] = np.log10(decision_contingency['complete']/decision_contingency['all'])
decision_contingency['Accepted vs. Complete'] = np.log10(decision_contingency['matriculated']/decision_contingency['complete'])
decision_contingency['Denied vs. Complete'] = np.log10(decision_contingency['deny']/decision_contingency['complete'])
decision_melt = decision_contingency.melt(id_vars=[feature], value_vars = ['all','incomplete','complete','deny','matriculated','Incomplete vs. All','Complete vs. All','Denied vs. Complete','Accepted vs. Complete'] )

decision_melt = decision_melt[decision_melt[feature] != 'total count']
decision_melt[title_feature] = decision_melt[feature].apply(lambda row: row.replace('_',' ').title() if isinstance(row,str) else row)
sorted_hue = list(decision_melt[title_feature].value_counts().sort_index().index)

fig, ax = plt.subplots(2,1,figsize=(10,20), sharey = True)

ax0 = sns.barplot(x="variable",
                  y="value", 
                  hue=title_feature, 
                  hue_order = sorted_hue,
                  data= decision_melt[decision_melt.variable.isin(['Incomplete vs. All','Complete vs. All'])],
                  ax = ax[0],
                  palette = palette_color)


ax0.legend_.remove()
ax0.set_title('Conditioned on All Applications')
ax0.set_xlabel('')
ax0.set_ylabel('Log Fold Change')

ax1 = sns.barplot(x="variable",
                  y="value",
                  hue=title_feature,
                  hue_order = sorted_hue,
                  data= decision_melt[decision_melt.variable.isin(['Accepted vs. Complete','Denied vs. Complete'])],
                  ax = ax[1],
                  palette = palette_color)

ax1.legend_.remove()
ax1.set_title('Conditioned on Complete Applications')
ax1.set_xlabel('')
ax1.set_ylabel('')
sns.despine()
# plt.suptitle('Changes in {} Proportion Ditributions Across Application Status'.format(title_feature),size = 20)
plt.close()#plt.show() 

In [17]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    # did_df_deny,
                    did_df_matriculated,
                    ]

outcomes_title_list = ['All Started Applications',
                      'Incomplete Applications',
                      'Complete Applications',
                      # 'Denied Applications',
                      'Accepted Applications',
                      ]
fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

axes = np.ravel(ax)
attribute = 'std_sex'
title_atribute = 'Sex'
legend_labels = []
for i,ax in enumerate(axes):
  working_df = outcomes_df_list[i]

  working_df = working_df.sort_values(attribute)
  values_series = working_df[attribute].value_counts(dropna = False).sort_index()
  values = np.array(values_series.values)
  labels = [i.replace('_',' ').title() if isinstance(i,str) else i for i in values_series.index ]


  explode = [.5 if value/values.sum() <.02 else 0 for value in values]
  pct_labels = [s + '%' if explode[i]==0 else '' for i,s in enumerate(list(np.round(values/values.sum()*100,1).astype(str)))]

  palette_color = sns.color_palette('pastel')
  mpl.rcParams['font.size'] = 10
  wedges, autotexts,  _= ax.pie(values, 
                              explode=explode,
                              colors = palette_color,
                              # labels = labels,
                              autopct=make_autopct(values)#pct_labels,#'%1.1f%%',                               
                              )

  title = '{}'.format(outcomes_title_list[i])
  ax.set_title(title)
  
  print(len(legend_labels))
  if len(labels)>len(legend_labels):
    # print(len(legend_labels))
    legend_labels = labels
    legend_wedges = wedges

plt.legend(legend_wedges, legend_labels,
          title=title_atribute,
          loc="lower right",
          bbox_to_anchor=(1, -.2, .2, 0)
          )
plt.close()#plt.show() 

0
3
3
3


In [18]:
outcome_series_list = []
feature = 'std_sex'
title_feature = 'Sex'
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

for df in outcomes_df_list:
  total_counts = pd.Series({'total count':df[feature].value_counts(dropna=False).sum()})
  normalized_counts = df[feature].value_counts(dropna=False)/df[feature].value_counts(dropna=False).sum()
  # normalized_counts = normalized_counts.append(total_counts)
  outcome_series_list.append(normalized_counts)

decision_contingency = pd.DataFrame(outcome_series_list,index = ['all','incomplete','complete','deny','matriculated']).T.reset_index()
decision_contingency = decision_contingency.rename(columns = {'index':feature})
decision_contingency['Incomplete vs. All'] = np.log10(decision_contingency['incomplete']/decision_contingency['all'])
decision_contingency['Complete vs. All'] = np.log10(decision_contingency['complete']/decision_contingency['all'])
decision_contingency['Accepted vs. Complete'] = np.log10(decision_contingency['matriculated']/decision_contingency['complete'])
decision_contingency['Denied vs. Complete'] = np.log10(decision_contingency['deny']/decision_contingency['complete'])
decision_melt = decision_contingency.melt(id_vars=[feature], value_vars = ['all','incomplete','complete','deny','matriculated','Incomplete vs. All','Complete vs. All','Denied vs. Complete','Accepted vs. Complete'] )

decision_melt = decision_melt[decision_melt[feature] != 'total count']

fig, ax = plt.subplots(2,1,figsize=(10,20), sharey = True)

ax0 = sns.barplot(x="variable",
                  y="value", 
                  hue=feature, 
                  data= decision_melt[decision_melt.variable.isin(['Incomplete vs. All','Complete vs. All'])],
                  ax = ax[0],
                  palette = palette_color)
ax0.set_title('Conditioned on All Applications')
ax0.set_xlabel('')
ax0.set_ylabel('Log Fold Change')

ax1 = sns.barplot(x="variable",
                  y="value",
                  hue=feature,
                  data= decision_melt[decision_melt.variable.isin(['Accepted vs. Complete','Denied vs. Complete'])],
                  ax = ax[1],
                  palette = palette_color)


ax1.set_title('Conditioned on Complete Applications')
ax1.set_xlabel('')

# plt.suptitle('Changes in {} Proportion Ditributions Across Application Status'.format(title_feature),size = 20)
plt.close()#plt.show() 

In [19]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    # did_df_deny,
                    did_df_matriculated,
                    ]

outcomes_title_list = ['All Started Applications',
                      'Incomplete Applications',
                      'Complete Applications',
                      # 'Denied Applications',
                      'Accepted Applications',
                      ]
fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

axes = np.ravel(ax)
attribute = 'std_hispanic'
title_atribute = 'Hispanic Identification'
legend_labels = []
for i,ax in enumerate(axes):
  working_df = outcomes_df_list[i]

  working_df = working_df.sort_values(attribute)
  values_series = working_df[attribute].value_counts(dropna = False).sort_index()
  values = np.array(values_series.values)
  labels = [i.replace('_',' ').title() if isinstance(i,str) else i for i in values_series.index ]


  explode = [.5 if value/values.sum() <.02 else 0 for value in values]
  pct_labels = [s + '%' if explode[i]==0 else '' for i,s in enumerate(list(np.round(values/values.sum()*100,1).astype(str)))]

  palette_color = sns.color_palette('pastel')
  mpl.rcParams['font.size'] = 10
  wedges, autotexts,  _= ax.pie(values, 
                              explode=explode,
                              colors = palette_color,
                              # labels = labels,
                              autopct=make_autopct(values)#pct_labels,#'%1.1f%%',                               
                              )

  title = '{}'.format(outcomes_title_list[i])
  ax.set_title(title)
  
  print(len(legend_labels))
  if len(labels)>len(legend_labels):
    # print(len(legend_labels))
    legend_labels = labels
    legend_wedges = wedges

plt.legend(legend_wedges, legend_labels,
          title=title_atribute,
          loc="lower right",
          bbox_to_anchor=(1, -.2, .2, 0)
          )

plt.close()#plt.show() 

0
3
3
3


In [20]:
outcome_series_list = []
feature = 'std_hispanic'
title_feature = 'Hispanic'
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

for df in outcomes_df_list:
  total_counts = pd.Series({'total count':df[feature].value_counts(dropna=False).sum()})
  normalized_counts = df[feature].value_counts(dropna=False)/df[feature].value_counts(dropna=False).sum()
  # normalized_counts = normalized_counts.append(total_counts)
  outcome_series_list.append(normalized_counts)

decision_contingency = pd.DataFrame(outcome_series_list,index = ['all','incomplete','complete','deny','matriculated']).T.reset_index()
decision_contingency = decision_contingency.rename(columns = {'index':feature})
decision_contingency['Incomplete vs. All'] = np.log10(decision_contingency['incomplete']/decision_contingency['all'])
decision_contingency['Complete vs. All'] = np.log10(decision_contingency['complete']/decision_contingency['all'])
decision_contingency['Accepted vs. Complete'] = np.log10(decision_contingency['matriculated']/decision_contingency['complete'])
decision_contingency['Denied vs. Complete'] = np.log10(decision_contingency['deny']/decision_contingency['complete'])
decision_melt = decision_contingency.melt(id_vars=[feature], value_vars = ['all','incomplete','complete','deny','matriculated','Incomplete vs. All','Complete vs. All','Denied vs. Complete','Accepted vs. Complete'] )

decision_melt = decision_melt[decision_melt[feature] != 'total count']

fig, ax = plt.subplots(2,1,figsize=(10,20), sharey = True)

ax0 = sns.barplot(x="variable",
                  y="value", 
                  hue=feature, 
                  data= decision_melt[decision_melt.variable.isin(['Incomplete vs. All','Complete vs. All'])],
                  ax = ax[0],
                  palette = palette_color)
ax0.set_title('Conditioned on All Applications')
ax0.set_xlabel('')
ax0.set_ylabel('Log Fold Change')

ax1 = sns.barplot(x="variable",
                  y="value",
                  hue=feature,
                  data= decision_melt[decision_melt.variable.isin(['Accepted vs. Complete','Denied vs. Complete'])],
                  ax = ax[1],
                  palette = palette_color)


ax1.set_title('Conditioned on Complete Applications')
ax1.set_xlabel('')

# plt.suptitle('Changes in {} Proportion Ditributions Across Application Status'.format(title_feature),size = 20)
plt.close()#plt.show() 

In [21]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    # did_df_deny,
                    did_df_matriculated,
                    ]

outcomes_title_list = ['All Started Applications',
                      'Incomplete Applications',
                      'Complete Applications',
                      # 'Denied Applications',
                      'Accepted Applications',
                      ]
fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))



axes = np.ravel(ax)
attribute = 'std_family_income_range'
title_atribute = 'Family Income Range'

ordered_values = pd.Series(["I don't know",
                            "Less Than $19,999",
                            "$20,000 To 39,999",
                            "$40,000 To 59,999",
                            "$60,000 To 79,999",
                            "$80,000 To 99,999",
                            "$100,000 To 149,999",
                            "More Than $150,000",
                            np.nan])

legend_labels = []
for i,ax in enumerate(axes):
  working_df = outcomes_df_list[i]

  working_df = working_df.sort_values(attribute)
  values_series = working_df[attribute].value_counts(dropna = False).sort_index()
  values = np.array(values_series.values)
  labels = [i.replace('_',' ').title() if isinstance(i,str) else i for i in values_series.index ]


  explode = [.5 if value/values.sum() <.03 else 0 for value in values]
  pct_labels = [s + '%' if explode[i]==0 else '' for i,s in enumerate(list(np.round(values/values.sum()*100,1).astype(str)))]

  palette_color = sns.color_palette('pastel')
  mpl.rcParams['font.size'] = 10
  wedges, autotexts,  _= ax.pie(values, 
                              explode=explode,
                              colors = palette_color,
                              # labels = labels,
                              autopct=make_autopct(values,thresh=3)#pct_labels,#'%1.1f%%',                               
                              )

  title = '{}'.format(outcomes_title_list[i])
  ax.set_title(title)
  
  print(len(legend_labels))
  if len(labels)>len(legend_labels):
    # print(len(legend_labels))
    legend_labels = labels
    legend_wedges = wedges

plt.legend(legend_wedges, legend_labels,
          title=title_atribute,
          loc="lower right",
          bbox_to_anchor=(1, -.2, .2, 0)
          )
plt.close()#plt.show() 

0
9
9
9


In [22]:
outcome_series_list = []
feature = 'std_family_income_range'
title_feature = 'Family Income'
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

for df in outcomes_df_list:
  total_counts = pd.Series({'total count':df[feature].value_counts(dropna=False).sum()})
  normalized_counts = df[feature].value_counts(dropna=False)/df[feature].value_counts(dropna=False).sum()
  # normalized_counts = normalized_counts.append(total_counts)
  outcome_series_list.append(normalized_counts.sort_index())

decision_contingency = pd.DataFrame(outcome_series_list,index = ['all','incomplete','complete','deny','matriculated']).T.reset_index()
decision_contingency = decision_contingency.rename(columns = {'index':feature})
decision_contingency['Incomplete vs. All'] = np.log10(decision_contingency['incomplete']/decision_contingency['all'])
decision_contingency['Complete vs. All'] = np.log10(decision_contingency['complete']/decision_contingency['all'])
decision_contingency['Accepted vs. Complete'] = np.log10(decision_contingency['matriculated']/decision_contingency['complete'])
decision_contingency['Denied vs. Complete'] = np.log10(decision_contingency['deny']/decision_contingency['complete'])
decision_melt = decision_contingency.melt(id_vars=[feature], value_vars = ['all','incomplete','complete','deny','matriculated','Incomplete vs. All','Complete vs. All','Denied vs. Complete','Accepted vs. Complete'] )

decision_melt = decision_melt[decision_melt[feature] != 'total count']
decision_melt[title_feature] = decision_melt[feature].apply(lambda row: row.replace('_',' ').title() if isinstance(row,str) else row)
sorted_hue = list(decision_melt[title_feature].value_counts().sort_index().index)

fig, ax = plt.subplots(2,1,figsize=(10,20), sharey = True)

ax0 = sns.barplot(x="variable",
                  y="value", 
                  hue=feature, 
                  data= decision_melt[decision_melt.variable.isin(['Incomplete vs. All','Complete vs. All'])],
                  ax = ax[0],
                  palette = palette_color)
ax0.set_title('Conditioned on All Applications')
ax0.set_xlabel('')
ax0.set_ylabel('Log Fold Change')

ax1 = sns.barplot(x="variable",
                  y="value",
                  hue=feature,
                  data= decision_melt[decision_melt.variable.isin(['Accepted vs. Complete','Denied vs. Complete'])],
                  ax = ax[1],
                  palette = palette_color)


ax1.set_title('Conditioned on Complete Applications')
ax1.set_xlabel('')

# plt.suptitle('Changes in {} Proportion Ditributions Across Application Status'.format(title_feature),size = 20)
plt.close()#plt.show() 

In [23]:
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    # did_df_deny,
                    did_df_matriculated,
                    ]

outcomes_title_list = ['All Started Applications',
                      'Incomplete Applications',
                      'Complete Applications',
                      # 'Denied Applications',
                      'Accepted Applications',
                      ]
fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))



axes = np.ravel(ax)
attribute = 'std_parent_education'
title_atribute = 'Parent College Education'

ordered_values = pd.Series(["I don't know",
                            "Less Than $19,999",
                            "$20,000 To 39,999",
                            "$40,000 To 59,999",
                            "$60,000 To 79,999",
                            "$80,000 To 99,999",
                            "$100,000 To 149,999",
                            "More Than $150,000",
                            np.nan])

legend_labels = []
for i,ax in enumerate(axes):
  working_df = outcomes_df_list[i]

  working_df = working_df.sort_values(attribute)
  values_series = working_df[attribute].value_counts(dropna = False).sort_index()
  values = np.array(values_series.values)
  labels = [i.replace('_',' ').title() if isinstance(i,str) else i for i in values_series.index ]


  explode = [.5 if value/values.sum() <.02 else 0 for value in values]
  pct_labels = [s + '%' if explode[i]==0 else '' for i,s in enumerate(list(np.round(values/values.sum()*100,1).astype(str)))]

  palette_color = sns.color_palette('pastel')
  mpl.rcParams['font.size'] = 10
  wedges, autotexts,  _= ax.pie(values, 
                              explode=explode,
                              colors = palette_color,
                              # labels = labels,
                              autopct=make_autopct(values)#pct_labels,#'%1.1f%%',                               
                              )

  title = '{}'.format(outcomes_title_list[i])
  ax.set_title(title)
  
  print(len(legend_labels))
  if len(labels)>len(legend_labels):
    # print(len(legend_labels))
    legend_labels = labels
    legend_wedges = wedges

plt.legend(legend_wedges, legend_labels,
          title=title_atribute,
          loc="lower right",
          bbox_to_anchor=(1, -.2, .2, 0)
          )

plt.close()#plt.show() 

0
4
4
4


In [24]:
outcome_series_list = []
feature = 'std_parent_education'
title_feature = 'Parent College Education'
outcomes_df_list = [did_df_working,
                    did_df_no_decision,
                    did_df_complete,
                    did_df_deny,
                    did_df_matriculated,
                    ]

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))
# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

# fig, ax = plt.subplots(2,2,figsize=(15, 15), subplot_kw=dict(aspect="equal"))

for df in outcomes_df_list:
  total_counts = pd.Series({'total count':df[feature].value_counts(dropna=False).sum()})
  normalized_counts = df[feature].value_counts(dropna=False)/df[feature].value_counts(dropna=False).sum()
  # normalized_counts = normalized_counts.append(total_counts)
  outcome_series_list.append(normalized_counts)

decision_contingency = pd.DataFrame(outcome_series_list,index = ['all','incomplete','complete','deny','matriculated']).T.reset_index()
decision_contingency = decision_contingency.rename(columns = {'index':feature})
decision_contingency['Incomplete vs. All'] = np.log10(decision_contingency['incomplete']/decision_contingency['all'])
decision_contingency['Complete vs. All'] = np.log10(decision_contingency['complete']/decision_contingency['all'])
decision_contingency['Accepted vs. Complete'] = np.log10(decision_contingency['matriculated']/decision_contingency['complete'])
decision_contingency['Denied vs. Complete'] = np.log10(decision_contingency['deny']/decision_contingency['complete'])
decision_melt = decision_contingency.melt(id_vars=[feature], value_vars = ['all','incomplete','complete','deny','matriculated','Incomplete vs. All','Complete vs. All','Denied vs. Complete','Accepted vs. Complete'] )

decision_melt = decision_melt[decision_melt[feature] != 'total count']
decision_melt[title_feature] = decision_melt[feature].apply(lambda row: row.replace('_',' ').title() if isinstance(row,str) else row)
sorted_hue = list(decision_melt[title_feature].value_counts().sort_index().index)

fig, ax = plt.subplots(2,1,figsize=(10,20), sharey = True)

ax0 = sns.barplot(x="variable",
                  y="value", 
                  hue=title_feature, 
                  hue_order = sorted_hue,
                  data= decision_melt[decision_melt.variable.isin(['Incomplete vs. All','Complete vs. All'])],
                  ax = ax[0],
                  palette = palette_color)


ax0.legend_.remove()
ax0.set_title('Conditioned on All Applications')
ax0.set_xlabel('')
ax0.set_ylabel('Log Fold Change')

ax1 = sns.barplot(x="variable",
                  y="value",
                  hue=title_feature,
                  hue_order = sorted_hue,
                  data= decision_melt[decision_melt.variable.isin(['Accepted vs. Complete','Denied vs. Complete'])],
                  ax = ax[1],
                  palette = palette_color)

ax1.legend_.remove()
ax1.set_title('Conditioned on Complete Applications')
ax1.set_xlabel('')
ax1.set_ylabel('')
sns.despine()
# plt.suptitle('Changes in {} Proportion Ditributions Across Application Status'.format(title_feature),size = 20)
plt.close()#plt.show() 